In [ ]:
# 파일 불러와서 (비식별 전) 화면에 출력까지 됨
# 총계처리 시도중
# 선택하면 1로 바꿈

In [1]:
import pandas as pd
import numpy as np
import random
import sys
import pandas as pd #import csv
from PyQt5.QtWidgets import QApplication, QWidget, QPushButton, QGridLayout, QFileDialog, QLabel
from PyQt5.QtWidgets import *
from PyQt5.QtGui import *
from PyQt5 import uic, QtGui

In [2]:
def masking(self,column,direction,number):           #(데이터프레임, 처리할 칼럼, 남겨둘 정보의 위치, 남겨둘 정보의 개수)
    self.backup = self[column]
    before_data=self.loc[:,column]                   #columns 가져오기
    
    after = self.astype({column : 'object'})           #columns 데이터 타입 변경
    
    for i in range(len(before_data.index)):
        string = str(before_data.iloc[i])
        length = len(string)                  #전체 문자 길이 구하기
        m_length = length - number            #마스킹할 문자 길이 구하기
        
        if number > length:                                                           #남겨둘 문자의 길이가 전체 문자길이보다 작을 경우 
            string = string.replace(string[:],"*"*length)                             # ex) 전체 문자길이=2, 남겨둘 문자=4이면 전체 문자길이가 더 작아 마스킹이 안되므로 전체 마스킹처리
        
        if direction == 'right':              #남겨둘 정보(마스킹하지 않을 정보)의 위치=오른쪽
            if number <= length:                                                      #남겨둘 문자의 길이가 전체 문자길이보다 작을 경우
                string = string.replace(string[:-number],"*"*m_length)             #남겨둘 문자를 제외한 모든 문자 "*"로 마스킹처리
                
        elif direction == 'left':             #남겨둘 정보(마스킹하지 않을 정보)의 위치=왼쪽
            if number <= length:
                string = string.replace(string[number:],"*"*m_length)                  #남겨둘 문자를 제외한 모든 문자 "*"로 마스킹처리
        after.at[i,column] = string
        
    return after            

In [ ]:
form_class = uic.loadUiType("third_JSM.ui")[0]

#화면을 띄우는데 사용되는 Class 선언
class WindowClass(QMainWindow, form_class) :    
    def __init__(self) :
        super().__init__()
        self.setupUi(self)
        
        # 로고 이미지
        qPixmapVar = QPixmap()
        qPixmapVar.load("logo2.jpg")
        self.Logo.setPixmap(qPixmapVar)
        
        self.open_button.clicked.connect(self.open_file) # open_button 클릭 관련
        self.start_button.clicked.connect(self.start) # '비식별 처리' start 버튼 클릭 곤련
        self.save_button.clicked.connect(self.save_file) # save_button 클릭 관련
        self.masking_button.clicked.connect(self.Masking) # '비식별 처리' Masking 버튼 클릭 곤련
        
    def open_file(self) : # 파일 불러오기
        open_name = QFileDialog.getOpenFileName(self, 'Open File', './')        
        data = pd.read_csv(open_name[0], encoding='euc-kr')
        
        global after # 비식별 처리 후 데이터를 저장할 전역변수 after
        after = data
        
        # table_1 widget 행, 열 추가
        self.table_1.setRowCount(len(data))
        self.table_1.setColumnCount(len(list(data)))
        self.table_1.setHorizontalHeaderLabels(list(data))

        # table_1 widget에 불러온 파일의 데이터 추가
        for i in range (len(data)) :
            for j in range (len(list(data))) :
                self.table_1.setItem(i, j, QTableWidgetItem(format(data.values.tolist()[i][j])))
    
    def start(self) : # 임시 총계처리 평균
        row = self.table_1.currentRow()
        column = self.table_1.currentColumn()
        
        after[after.columns[column]] = after[after.columns[column]].mean() # 선택한 열 평균으로 총계처리
        
        for i in range (len(after)) :
            self.table_1.setItem(i, column, QTableWidgetItem(format(after.values.tolist()[i][column]))) # 비식별한 열 출력
   
    def Masking(self) :  # 마스킹 처리
        row = self.table_1.currentRow()
        column = self.table_1.currentColumn()
        tmp = masking(after,after.columns[column],'left',2)
        
        for i in range (len(tmp)) :
            self.table_1.setItem(i, column, QTableWidgetItem(format(tmp.values.tolist()[i][column]))) # 비식별한 열 출력
            

    def save_file(self) : # 비식별처리 후 파일 저장
        save_name = QFileDialog.getSaveFileName(self, 'Open File', './')
        after.to_csv(save_name[0], encoding='euc-kr', index=False)
        
if __name__ == "__main__" :
    app = QApplication(sys.argv) 
    myWindow = WindowClass() 
    myWindow.show()
    app.exec_()